In [1]:


from astropy import units as u
from astropy import time

import numpy as np

from poliastro import iod
from poliastro.bodies import Mars, Earth, Jupiter, Sun
from poliastro.ephem import Ephem
from poliastro.maneuver import Maneuver
from poliastro.twobody import Orbit
from poliastro.util import time_range
from poliastro.plotting import OrbitPlotter3D, StaticOrbitPlotter
import math
import matplotlib.pyplot as plt
# More info: https://plotly.com/python/renderers/
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

from astropy.coordinates import solar_system_ephemeris


solar_system_ephemeris.set("jpl")

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.dates as mdates

c:\Users\jacki\miniforge-pypy3\envs\python_3.9\lib\site-packages\poliastro\core\_jit.py:29: UserWarning: Could not import numba package. All poliastro functions will work properly but the CPU intensive algorithms will be slow. Consider installing numba to boost performance.
  warnings.warn(


In [3]:
#Porkchop plot v2
#Brute force lambert solve for 5 day intervals (thus 5/~3000 = 600 timesteps, for up to 1 year (?) thus 40000 lambert solves)
tof_resolution = 5*u.day
max_tof = 365*u.day

time_resolution = 5*u.day
start_date = time.Time("2030-01-01 00:01", scale="utc").tdb
end_date = time.Time("2032-12-31 23:59", scale="utc").tdb

#end_date = time.Time("2039-12-31 23:59", scale="utc").tdb


#Body_1 to Body_2 on top, other on bottom
body_1 = Earth
body_2 = Mars


ephem_1 = Ephem.from_body(body_1, time_range(start_date, end=end_date+max_tof))
ephem_2 = Ephem.from_body(body_2, time_range(start_date, end=end_date+max_tof))

delta_time = end_date-start_date
dv_array = np.empty((rows, cols), dtype=object)

for i in range(int((delta_time/time_resolution).value)):
    #print("jeff")
    for j in range(1,int((max_tof/tof_resolution).value)):
        t_launch = start_date+i*time_resolution
        t_arrival = start_date+i*time_resolution+j*tof_resolution
        ss_planet_1 = Orbit.from_ephem(Sun, ephem_1, t_launch)
        ss_planet_2 = Orbit.from_ephem(Sun, ephem_2, t_arrival)
        # print(t_launch)
        # print(t_arrival)
        # Solve for the transfer maneuver
        man_lambert = Maneuver.lambert(ss_planet_1, ss_planet_2)
        ejection = man_lambert[0]
        dv = np.linalg.norm(ejection[1].value)

#convergance time needs to be better for website - ~40 seconds/yr
#perhaps can pre-compute plots for major planets



NameError: name 'rows' is not defined

In [ ]:
#plotting
# Convert Astropy Time to Matplotlib dates
plt_start_date = mdates.date2num(start_date)
plt_end_date = mdates.date2num(end_date)


# Convert to a float and replace None with 0 (or another default value) cause non solutions go null in that array
clean_data = np.nan_to_num(dv_array.astype(float)).transpose()


num_dates = 5
dates = np.linspace(start_date, end_date, num_dates)
date_labels = [mdates.num2date(date).strftime('%Y-%m-%d') for date in dates]
date_ticks = np.linspace(0, clean_data.shape[1] - 1, num_dates)
y_ticks = np.linspace(0, clean_data.shape[0] - 1, 5)  # 4 quarters + start of the year
y_labels = ['0 Yr', '0.25 Yr', '0.5 Yr', '0.75 Yr', '1 Yr']



# Define a custom colormap from blue to red for the porkchop plot
#TBA proper coloring scheme!
#
cmap = colors.LinearSegmentedColormap.from_list("red_to_blue", ["red", "blue"])


# Create the plot
#print(dv_array[20,30])
plt.imshow(clean_data, cmap=cmap, aspect='auto', vmin=0, vmax=20000,origin = 'lower')
# Set the x-ticks to correspond to the dates
plt.xticks(ticks=date_ticks, labels=date_labels, rotation=45)
plt.yticks(ticks=y_ticks, labels=y_labels)

print(dv_array[60][13])
print(date_array[60][13])


# Add a colorbar to show the mapping from data values to colors
plt.colorbar()

# Show the plot
plt.show()



